[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/BPR.ipynb)

# Bayesian Personalized Ranking(BPR)

In [ ]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [4]:
# 因子数
factors = 10
# 評価数の閾値
minimum_num_rating = 0
# エポック数
n_epochs = 50

In [6]:
# 行列分解用に行列を作成する
filtered_movielens_train = movielens_train.groupby("movie_id").filter(
    lambda x: len(x["movie_id"]) >= minimum_num_rating
)

movielens_train_high_rating = filtered_movielens_train[filtered_movielens_train.rating >= 4]

# 行列のインデックスと映画/ユーザーを対応させる辞書を作成
unique_user_ids = sorted(movielens_train_high_rating.user_id.unique())
unique_movie_ids = sorted(movielens_train_high_rating.movie_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
movie_id2index = dict(zip(unique_movie_ids, range(len(unique_movie_ids))))

In [8]:
from scipy.sparse import lil_matrix
# スパース行列を初期化して、各セルに値を入れていく
movielens_matrix = lil_matrix((len(unique_movie_ids), len(unique_user_ids)))
for i, row in movielens_train_high_rating.iterrows():
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    movielens_matrix[movie_index, user_index] = 1.0


In [ ]:
!pip install implicit==0.4.4

In [ ]:
# colab上で、implicitライブラリの実行時にエラーが出る場合は、「ランタイム→ランタイムのタイプを変更」でハードウェア アクセラレータとしてGPUを選択してください

In [10]:
import implicit

# モデルの初期化
model = implicit.bpr.BayesianPersonalizedRanking(factors=factors, iterations=n_epochs)

In [11]:
# 学習
model.fit(movielens_matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
from collections import defaultdict

# 推薦
recommendations = model.recommend_all(movielens_matrix.T)
pred_user2items = defaultdict(list)
for user_id, user_index in user_id2index.items():
    movie_indexes = recommendations[user_index, :]
    for movie_index in movie_indexes:
        movie_id = unique_movie_ids[movie_index]
        pred_user2items[user_id].append(movie_id)
pred_user2items

  0%|          | 0/997 [00:00<?, ?it/s]

defaultdict(list,
            {1: [733, 780, 597, 150, 500, 736, 595, 653, 339, 457],
             2: [593, 356, 296, 589, 457, 318, 1196, 480, 50, 1],
             3: [2858, 2571, 356, 4993, 2762, 4226, 2959, 1704, 3578, 3996],
             4: [733, 377, 356, 457, 780, 597, 736, 185, 292, 380],
             5: [1084, 800, 908, 1193, 1252, 194, 296, 1267, 36, 913],
             6: [1240, 593, 589, 1036, 2762, 1214, 2858, 1610, 1210, 2716],
             7: [750, 1080, 924, 1204, 1136, 1199, 1247, 1206, 2076, 858],
             8: [4993,
              5952,
              4262,
              5679,
              5445,
              5989,
              32587,
              33794,
              5378,
              51662],
             9: [1196, 260, 2571, 858, 541, 1136, 1214, 608, 1198, 593],
             10: [1271, 1084, 1247, 58, 2396, 1207, 534, 246, 261, 272],
             11: [1197, 2797, 2640, 2407, 2100, 2080, 1079, 919, 2081, 914],
             12: [2571, 356, 2329, 2028, 2706, 4963

In [16]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [17]:
# user_id=2に対するおすすめ(593, 356, 296)
movies[movies.movie_id.isin([593, 356, 296])]

,movie_id,title,genre,tag
293,296,Pulp Fiction (1994),"[Comedy, Crime, Drama]","[quotable, samuel l. jackson, quentin tarantin..."
352,356,Forrest Gump (1994),"[Comedy, Drama, Romance, War]","[psychology, tom hanks, seen more than once, l..."
587,593,"Silence of the Lambs, The (1991)","[Crime, Horror, Thriller]","[based on a book, anthony hopkins, demme, psyc..."
